# Predictor submission


The COVID-19 crisis is proving to be one of the world’s most critical challenges — a challenge bigger than any one government or organization can tackle on its own. Right now, countries around the world are not equipped to implement health and safety interventions and policies that effectively protect both their citizens and economies.
 
In order to fight this pandemic, we need access to localized, data-driven planning systems and the latest in artificial intelligence (AI) to help decision-makers develop and implement robust Intervention Plans (IPs) that successfully reduce infection cases and minimize economic impact.

**Intervention Plan (IP)**: A plan of action or schedule for setting and resetting various intervention policies at various strengths or stringency.

**Predictor Model**: Given a time sequence of IPs in effect, and other data like a time sequence of number of cases, a predictor model will estimate the number of cases in the future.

## Intervention Plan

An intervention plan consists of a set of [containment and closure policies](https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md#containment-and-closure-policies), as well as [health system policies](https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md#health-system-policies). Checkout the links to understand what these policies correspond to and how they are coded.

For instance the **C1_School closing** policy, which records closings of schools and universities, is coded like that:

| Code      | Meaning     |
| :-------- | :---------- |
|  0        | no measures |
|  1        | recommend closing|
|  2        | require closing (only some levels or categories, eg just high school, or just public schools) |
|  3        | require closing all levels |
| Blank     | no data |

Interventions plans are recorded daily for each countries and sometimes for regions. For this competition, the following policies are considered:

In [1]:
IP_COLUMNS = ['C1_School closing',
              'C2_Workplace closing',
              'C3_Cancel public events',
              'C4_Restrictions on gatherings',
              'C5_Close public transport',
              'C6_Stay at home requirements',
              'C7_Restrictions on internal movement',
              'C8_International travel controls',
              'H1_Public information campaigns',
              'H2_Testing policy',
              'H3_Contact tracing',
              'H6_Facial Coverings']

## Data
The university of Oxford Blavatnik School of Government is [tracking coronavirus government responses](https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker). They have assembled a [data set](https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv) containing historical data since January 1st, 2020 for the number of cases and IPs for most countries in the world.

In [2]:
import pandas as pd

In [3]:
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df = pd.read_csv(DATA_URL,
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [4]:
df.sample(3)

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
127036,El Salvador,SLV,NaN,NaN,NAT_TOTAL,2020-01-29,0.0,NaN,0.0,NaN,...,11.11,11.11,14.29,14.29,6.25,6.25,7.14,7.14,0.0,0.0
160749,United States,USA,New Jersey,US_NJ,STATE_TOTAL,2020-10-15,2.0,0.0,2.0,1.0,...,55.56,55.56,63.81,63.81,60.42,60.42,60.12,60.12,62.5,62.5
173563,Vietnam,VNM,NaN,NaN,NAT_TOTAL,2020-03-02,3.0,1.0,0.0,NaN,...,47.22,47.22,58.33,58.33,33.85,33.85,38.69,38.69,0.0,0.0


### Listing the number of cases and IPs

In [5]:
CASES_COLUMNS = ["CountryName", "RegionName", "Date", "ConfirmedCases"]

In [6]:
df[CASES_COLUMNS + IP_COLUMNS].sample(3)

,CountryName,RegionName,Date,ConfirmedCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
6206,Burundi,NaN,2021-06-20,5121.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
20337,Brazil,Mato Grosso,2021-05-07,371781.0,3.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,4.0
1162,Angola,NaN,2020-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Computing the daily change in cases
The **ConfirmedCases** column reports the total number of cases since the beginning of the epidemic for each country, region and day. From this number we can compute the daily change in confirmed cases by doing:

\begin{equation*}
DailyChangeConfirmedCases_t = ConfirmedCases_t - ConfirmedCases_{t-1}
\end{equation*}

Like this:

In [7]:
df["DailyChangeConfirmedCases"] = df.groupby(["CountryName", "RegionName"]).ConfirmedCases.diff().fillna(0)

### Listing the latest historical daily new cases for a given country and region
For instance, for country **United States**, region **California**, the latest available changes in confirmed cases are:

In [8]:
country = "United States"
region = "California"
country_region_df = df[(df.CountryName == country) & (df.RegionName == region)]
country_region_df[["CountryName", "RegionName", "Date", "ConfirmedCases", "DailyChangeConfirmedCases"]].tail(7)

,CountryName,RegionName,Date,ConfirmedCases,DailyChangeConfirmedCases
145712,United States,California,2021-07-14,3847746.0,2566.0
145713,United States,California,2021-07-15,3849540.0,1794.0
145714,United States,California,2021-07-16,3855167.0,5627.0
145715,United States,California,2021-07-17,3858391.0,3224.0
145716,United States,California,2021-07-18,3860499.0,2108.0
145717,United States,California,2021-07-19,3865655.0,5156.0
145718,United States,California,2021-07-20,NaN,0.0


## Predictor input
The goal of a predictor is to predict the expected number of daily cases for countries and regions for a list of days, assumging the given daily IPs have been or will be in place:

In [10]:
EXAMPLE_INPUT_FILE = "covid_xprize/validation/data/2020-09-30_historical_ip.csv"
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  dtype={"RegionName": str},
                                  encoding="ISO-8859-1")
prediction_input_df.tail()

,CountryName,RegionName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
64659,Zimbabwe,NaN,2020-09-26,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
64660,Zimbabwe,NaN,2020-09-27,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
64661,Zimbabwe,NaN,2020-09-28,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
64662,Zimbabwe,NaN,2020-09-29,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0
64663,Zimbabwe,NaN,2020-09-30,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0


In [12]:
prediction_input_df.columns

Index(['CountryName', 'RegionName', 'Date', 'C1_School closing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C5_Close public transport',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6_Facial Coverings'],
      dtype='object')

## Predictor expected output
The output produced by the predictor should look like that:

In [13]:
EXAMPLE_OUTPUT_FILE = "2020-08-01_2020-08-04_predictions_example.csv"
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")
prediction_output_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases,IsSpecialty
0,Aruba,NaN,2020-08-01,0.820071,0
1,Aruba,NaN,2020-08-02,0.872854,0
2,Aruba,NaN,2020-08-03,0.000000,0
3,Aruba,NaN,2020-08-04,0.000000,0
4,Afghanistan,NaN,2020-08-01,80.590128,0


### Specialty regions
In this output file example, Portugal and Spain have been marked as 'specialty' regions
by turning the flag to 1 in the optional 'IsSpecialty' column:

In [14]:
prediction_output_df[prediction_output_df.IsSpecialty == 1]

,CountryName,RegionName,Date,PredictedDailyNewCases,IsSpecialty
160,Spain,NaN,2020-08-01,458.737283,1
161,Spain,NaN,2020-08-02,408.084246,1
162,Spain,NaN,2020-08-03,6723.669282,1
163,Spain,NaN,2020-08-04,2142.191374,1
424,Portugal,NaN,2020-08-01,291.295733,1
425,Portugal,NaN,2020-08-02,242.627956,1
426,Portugal,NaN,2020-08-03,190.392662,1
427,Portugal,NaN,2020-08-04,118.052736,1


## Train a model
Train a predictor model that can produce the output file given the input file.

Use additional data sources if needed.

Predictors have to predict for all requested regions. Note that a predictor submission can consist of multiple models, for example those specializing in different regions, that are accessed through a __single__ `predict.py` call. A predictor must return a prediction in less than 1 hour for up to 180 days of prediction for up to 300 regions.

In [10]:
import os
#import pandas as pd
import requests

In [11]:
# WRITE YOUR CODE HERE
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
DATA_FILE = 'data/OxCGRT_latest.csv'

# Download the data set
data = requests.get(DATA_URL)

# Persist the data set locally in order to use it after submission to make predictions,
# as the sandbox won't have access to the internet anymore.
if not os.path.exists('data'):
    os.mkdir('data')
open(DATA_FILE, 'wb').write(data.content)

42451489

In [12]:
# Reload the module to get the latest changes
import valencia_predictor
from importlib import reload
reload(valencia_predictor)
from valencia_predictor import ValenciaPredictor

In [13]:
predictor = ValenciaPredictor(None, DATA_FILE)

TypeError: __init__() takes 1 positional argument but 3 were given

In [ ]:
%%time
predictor_model = predictor.train()

In [ ]:
# It's a good idea to save the trained model so it can be used to make predictions in the future
if not os.path.exists('models'):
    os.mkdir('models')
predictor_model.save_weights("models/trained_test1_model_weights.h5")

In [ ]:
# For examples, check out the covid_xprize/examples/predictors folder:
# - covid_xprize/examples/predictors/linear/Example-Train-Linear-Rollout-Model.ipynb
# - covid_xprize/examples/predictors/lstm/Example-LSTM-Predictor.ipynb

## Make predictions
Making predictions means saving a .csv file to the path specified in `output_file_path`.



In [ ]:
def predict(start_date: str,
            end_date: str,
            path_to_ips_file: str,
            output_file_path) -> None:
    """
    Generates and saves a file with daily new cases predictions for the given countries, regions and intervention
    plans, between start_date and end_date, included.
    :param start_date: day from which to start making predictions, as a string, format YYYY-MM-DDD
    :param end_date: day on which to stop making predictions, as a string, format YYYY-MM-DDD
    :param path_to_ips_file: path to a csv file containing the intervention plans between inception date (Jan 1 2020)
     and end_date, for the countries and regions for which a prediction is needed
    :param output_file_path: path to file to save the predictions to
    :return: Nothing. Saves the generated predictions to an output_file_path CSV file
    with columns "CountryName,RegionName,Date,PredictedDailyNewCases"
    """
    # !!! YOUR CODE HERE !!!
    raise NotImplementedError

In [ ]:
start_date = "2020-08-01"
end_date = "2020-08-04"
output_file = "my_first_predictions.csv"
predict(start_date, end_date, EXAMPLE_INPUT_FILE, output_file)

## Display predictions
If prediction worked ok, it generated the `output_file` file, that we can read like that:

In [ ]:
prediction_output_df = pd.read_csv(output_file,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")
prediction_output_df.head()

# Submission
Update `predict.py` to call the trained predictor and generate a predictor file.

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [1]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip covid_xprize/validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [2]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases,PredictedDailyNewRatios
Aruba,,2020-08-01,0,0.9998891353607178
Aruba,,2020-08-02,0,1.000319242477417
Aruba,,2020-08-03,0,1.0014559030532837
Aruba,,2020-08-04,0,1.0045603513717651
Afghanistan,,2020-08-01,38,0.9877288341522217
Afghanistan,,2020-08-02,130,1.01540207862854
Afghanistan,,2020-08-03,82,0.9672622680664062
Afghanistan,,2020-08-04,105,1.0014526844024658
Angola,,2020-08-01,50,1.054331660270691


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [3]:
# Check the pediction file is valid
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [4]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="covid_xprize/validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="covid_xprize/validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
import os
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = "covid_xprize/validation/tests/fixtures/OxCGRT_latest.csv"
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
os.makedirs(os.path.dirname(scenario_file), exist_ok=True)
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")